In [1]:
!pip install analitico>=2019.08.12.dev6
#!pip install --index-url https://test.pypi.org/simple/ analitico==2019.08.12.dev4

import os
import requests
import tempfile
import analitico
import simplejson as json

from analitico import logger
from analitico.utilities import time_ms

MB_SIZE = 1024 * 1024

You are using pip version 19.0.3, however version 19.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# calls are enabled using a developer token
sdk = analitico.authorize_sdk("tok_demo1_croJ7gVp4cW9", endpoint="http://127.0.0.1:8000/api/")
sdk

In [3]:
# test dataset with lots of large files
ds = sdk.get_dataset("ds_5je37sn4")
ds

In [ ]:
# random bytes to avoid compression, etc
size = 8 * MB_SIZE
data1 = bytearray(os.urandom(size))

# upload data directly to item's storage
with tempfile.NamedTemporaryFile() as f1:
  try:
    f1.write(data1)
    started_ms = time_ms()
    ds.upload(filepath=f1.name, remotepath="deleteme.data", direct=False)

    elapsed_ms = max(1, time_ms(started_ms))
    kb_sec = (size / 1024.0) / (elapsed_ms / 1000.0)
    msg = f"sdk upload: {size / MB_SIZE} MB in {elapsed_ms} ms, {kb_sec:.0f} KB/s"
    logger.info(msg)
    print(msg)
    
  except Exception as exc:
    print(exc)

In [ ]:
# random bytes to avoid compression, etc
size = 8 * MB_SIZE
data1 = bytearray(os.urandom(size))


file_url = ds.url + "/files/delete2.data"

with tempfile.NamedTemporaryFile() as f1:
    f1.write(data1)
    f1.seek(0)
    started_ms = time_ms()
    
    # multipart encoded upload
    headers = {"Authorization": "Bearer " + ds.sdk.token}
    response = requests.put(file_url, files={"file": f1}, headers=headers)
    print(response.status_code)    
    
    elapsed_ms = max(1, time_ms(started_ms))
    kb_sec = (size / 1024.0) / (elapsed_ms / 1000.0)
    msg = f"sdk upload (direct): {size / MB_SIZE} MB in {elapsed_ms} ms, {kb_sec:.0f} KB/s"
    logger.info(msg)
    
    if response.content:
        json = json.dumps(json.loads(str(response.content)), indent=2)
        print(json)